In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.signal as sig

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In the following block I have loaded a simulated spikes train and tried to get its low-pass filter as a calcium signal. the time constant here is chosen as tau=100ms.

In [3]:
spikes = np.load("spikes-10e4-ms.npy")

N = np.shape(spikes)[0]
wup_time = 1000
spikes = spikes[:, wup_time:]
sim_dur = np.shape(spikes)[1]

ns = np.random.normal(0, 1, (N, sim_dur))
noisy_sp = spikes + ns

calcium = np.zeros((N, sim_dur))
calcium_nsp = np.zeros((N, sim_dur))
tau = 100
dt = 1
const_A = np.exp((-1/tau)*dt)

calcium[:, 0] = spikes[:, 0]
calcium_nsp[:, 0] = spikes[:, 0]

for t in range(1, sim_dur):
    calcium[:, t] = const_A*calcium[:, t-1] + spikes[:, t]


for t in range(1, sim_dur):
    calcium_nsp[:, t] = const_A*calcium_nsp[:, t-1] + noisy_sp[:, t]


Here I have chosen neuron number 500 and added a gaussian noise of N(0,1).\
In the plot:\
greem: simulated calcium\
blue: clacium with noise\
orange: gaussian noiese\
red: spikes train

In [4]:
n = 500
#calciumN = calcium[n, :]
noise = np.random.normal(0,1, sim_dur)
cn = calcium[n, :] + noise

#calciumN_nsp = calcium_nsp[n, :]
#noise = np.random.normal(0,1, sim_dur)
cn_nsp = calcium_nsp[n, :] + noise

In [30]:
np.shape(cn)

(9000,)

In [4]:
np.save('noisy_cal',cn)

In [6]:
%matplotlib qt
plt.figure(1)
plt.subplot(2,1,1)
plt.title('spikes -> calcium + noise')
plt.plot(cn, label = 'calcium+noise')
plt.plot(noise, label = 'noise')
plt.plot(calcium[n, :], label = 'calcium')
plt.plot(spikes[n, :], label = 'spike')
plt.legend()

plt.subplot(2,1,2)
plt.title('spikes + noise -> calcium + noise')
plt.plot(cn_nsp, label = 'calcium+noise')
plt.plot(noise, label = 'noise')
plt.plot(calcium_nsp[n, :], label = 'calcium')
plt.plot(noisy_sp[n, :], label = 'spike')
plt.legend()

In [16]:
plt.figure(2)
plt.subplot(2,1,1)
plt.plot(cn)

plt.subplot(2,1,2)
plt.plot(cn_nsp)

Denoising by savitzky filter for signal and its first derivation and putting them into the exact solution of first order eq. works well. the only difference is the sign.

In [25]:
smooth_cal = sig.savgol_filter(cn, window_length=31, deriv=0, delta=1., polyorder=5)
smooth_deriv = sig.savgol_filter(cn, window_length=31, deriv=1, delta=1., polyorder=5)

#x = np. zeros(sim_dur)
#x[0] = 0.0



x = smooth_deriv + (1/tau)*smooth_cal

In [12]:
smooth_cal = sig.savgol_filter(cn_nsp, window_length=5, deriv=0, delta=1., polyorder=3)
smooth_deriv = sig.savgol_filter(cn_nsp, window_length=5, deriv=1, delta=1., polyorder=3)

#x = np. zeros(sim_dur)
#x[0] = 0.0



x = smooth_deriv + (1/tau)*smooth_cal

In [23]:
%matplotlib notebook
plt.plot(smooth_cal)

In [24]:
plt.plot(smooth_deriv)

 α ẏ(t) + β y(t)  =  x(t)

In [25]:

plt.subplot(2,1,1)
plt.plot(x)
plt.subplot(2,1,2)
plt.plot(cn)
#plt.plot(-x)

In [14]:
plt.subplot(3,1,1)
plt.plot(x)
plt.subplot(3,1,2)
plt.plot(noisy_sp[n, :])
plt.subplot(3,1,3)
plt.plot(spikes[n, :])

In [26]:
plt.subplot(2,1,1)
plt.plot(x)
plt.subplot(2,1,2)
plt.plot(spikes[500, :])

Error in callback <function _draw_all_if_interactive at 0x7f8052083be0> (for post_execute):


KeyboardInterrupt: 

I was thinking the resulted denoised signal can be percieved as calcium signal which I can deconvolve it again in order to gain spikes train. But it seems it doesn't work.

In [22]:
y = np. zeros(sim_dur)
y[0] = 0.0

for i in range(0, sim_dur-1):
    #y[i+1] = x[i+1] - const_A*x[i]
    y[i] = x[i] + const_A*x[i]

In [40]:
plt.plot(x)
#plt.plot(cn)
plt.plot(y)

In [25]:
np.corrcoef(spikes[500, :], y)

array([[1.        , 0.46656714],
       [0.46656714, 1.        ]])

Correlation of spikes train and resulted signal is 53% and is 33% with calcium signal(noisy or de-noised).

In [19]:
np.corrcoef(spikes[500, :], x)

array([[1.        , 0.55900041],
       [0.55900041, 1.        ]])

### Connectivity Inference

In [3]:
noiseN = np.random.normal(0,1, (N, sim_dur))
noisy_cal = calcium + noiseN

#np.save('noisy_cal_10e4', noisy_cal)

In [4]:
smooth_cal = sig.savgol_filter(noisy_cal, window_length=31, deriv=0, delta=1., polyorder=5)
smooth_deriv = sig.savgol_filter(noisy_cal, window_length=31, deriv=1, delta=1., polyorder=5)

#denoised noisy_calcium which is going to inferene procedure
signal = smooth_deriv + (1/tau)*smooth_cal

np.corrcoef(spikes.flatten(), signal.flatten())[0, 1]

0.5152649990295131

In [11]:
cum_spikes = np.cumsum(spikes[500, :])
cum_signal = np.cumsum(signal[500, :])

%matplotlib qt
plt.figure(2)
plt.plot(cum_signal)
plt.plot(cum_spikes)

In [10]:
#plt.figure()
plt.subplot(2,1,1)
plt.plot(signal[500, 1:8900])
plt.subplot(2,1,2)
plt.plot(spikes[500, 1:8900])

In [14]:
plt.plot(signal[500, 4000:5000], spikes[500, 4000:5000], '.')

In [20]:

G = np.load('connectivity-10e5-ms.npy')
G = G - (np.diag(np.diag(G)))

In [21]:
k = 10
Y = signal[:, k:]
Y_prime = signal[:, :-k]

yk = Y.T
y_k = Y_prime.T

np.shape(yk)

(98990, 1250)

In [22]:
reg = LinearRegression(n_jobs=-1).fit(y_k, yk)
a = reg.coef_
a = a - (np.diag(np.diag(a)))
np.corrcoef(G.flatten(), a.flatten())[0, 1]

0.5554727101379358

In [15]:
k = 10
Y = spikes[:, k:]
Y_prime = spikes[:, :-k]

yk = Y.T
y_k = Y_prime.T


reg = LinearRegression(n_jobs=-1).fit(y_k, yk)
a = reg.coef_
a = a - (np.diag(np.diag(a)))
np.corrcoef(G.flatten(), a.flatten())[0, 1]

0.40761319708918237

In [16]:
k = 10
Y = calcium[:, k:]
Y_prime = calcium[:, :-k]

yk = Y.T
y_k = Y_prime.T


reg = LinearRegression(n_jobs=-1).fit(y_k, yk)
a = reg.coef_
a = a - (np.diag(np.diag(a)))
np.corrcoef(G.flatten(), a.flatten())[0, 1]

0.48021036478665435